TBPS

In [1]:
# Needs Uni enviornment

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd

In [3]:
journal = 'tbps20'

In [4]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['BIM', 'Building Information Modelling']
# searchTerm2 = ['Green Building', 'Green Buildings']

# searchTerm1 = ['a']
# searchTerm2 = ['the']

tc = 0
noResultsList = []

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

baseUrl = 'https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22'
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        query = baseUrl + item1 +"%22+AND+%22"+ item2+"%22&Ppub=&startPage=&SeriesKey=" + journal
        print(query)
        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        allParas = soup.find_all("p")
        dd = allParas[0].text
        miniData = dd.split(' ')
        NumberOfResultsStr = miniData[2]
        indexOfcomma = NumberOfResultsStr.find(',')

        if indexOfcomma != -1:
            NumberOfResultsStr = NumberOfResultsStr.replace(',','')

        number = int(NumberOfResultsStr)
        tc = tc + number
                
        page = number // 10
        lastPage = number % 10
        if lastPage > 0:
            page = page + 1

        pageSize = page
#         print(pageSize)

        cnt = 0
        allLinks = []
        
#         for testing purpose only
#         pageSize = 2

        if number == 0:
            noResultsList.append((item1 +","+item2))
            print("No results found")
        else:
            


            print("Extracting all Links")

            for currentPage in range(0,pageSize):
                print("In page: ",currentPage)

                if currentPage != 0:
                    newQuery = query + '&pageSize=10&subjectTitle=&startPage=' + str(currentPage)
                    cnt = cnt + 1
                else:
                    newQuery = query


                print("Searching for query: ",newQuery)
                response = requests.get(newQuery)
                soup = BeautifulSoup(response.text, 'html.parser')
                mydivs = soup.findAll("a", {'class':['ref nowrap']})

                for d in mydivs:
                    url = 'https://www.tandfonline.com' + d.get('href')
                    allLinks.append(url)
    #             print(allLinks)

            print("Extracting data")

            for link in allLinks:
                df.loc[row,'searchTerm1'] = item1
                df.loc[row,'searchTerm2'] = item2
                df.loc[row,'url'] = link
                html_content = requests.get(link).text
                soup = BeautifulSoup(html_content, "lxml")

                for i in soup.find('span',{'class':['NLM_article-title' ,'hlFld-title']}):
    #                 print("Title: ",i)
                    df.loc[row,'Title'] = i

                temp = soup.findAll('h2',{'class':['section-heading-2']} )
                print("Number of sections: ",len(temp))            

                count = 0
                for i in range(len(temp)):

                    if temp[i].text == temp[i].find_next('p').text:

                        extract = "section" + str(count)
                        extractData = temp[i].find_next('p').find_next('p').text
                        df.loc[row,extract] = temp[i].text
                        sectionData = "sectionData" + str(count)                    
                        df.loc[row,sectionData] = extractData
                        count = count + 1
                    else:
                        extract = "section" + str(count)
                        extractData = temp[i].find_next('p').text
                        df.loc[row,extract] = temp[i].text
                        sectionData = "sectionData" + str(count)
                        df.loc[row,sectionData] = extractData
                        count = count + 1


                row = row + 1
                print("row: ",row)


['Green+Building', 'Green+Buildings', 'Sustainable+building', 'Sustainable+buildings', 'Environmentally+friendly+design', 'Sustainability', 'Sustainable+development', 'Energy', 'Water', 'Waste', 'Thermal+comfort', 'Electricity', 'Solar', 'Wind+power', 'Green+infrastructure', 'Life+cycle+assessment']
['BIM', 'Building+Information+Modelling', 'Building+information+model', '6D+BIM', 'Digital+construction', 'Smart+construction', 'Digital+engineering']
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Buildings%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Sustainable+building%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Su

No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Solar%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Wind+power%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Green+infrastructure%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Life+cycle+assessment%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%226D+BIM%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%226D+BIM%22+AND+%22Green+Buildings%22&Ppub=&startPage=&Series

No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Green+infrastructure%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Life+cycle+assessment%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+engineering%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+engineering%22+AND+%22Green+Buildings%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+engineering%22+AND+%22Sustainable+building%22&Ppub=&startPage=&SeriesKey=tbps20
No results found
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+engineering%22+AND+%22Sustainable+buildings%22&P

In [6]:
noResultsList

['BIM,Green+Building',
 'BIM,Green+Buildings',
 'BIM,Sustainable+building',
 'BIM,Sustainable+buildings',
 'BIM,Environmentally+friendly+design',
 'BIM,Sustainability',
 'BIM,Sustainable+development',
 'BIM,Water',
 'BIM,Waste',
 'BIM,Thermal+comfort',
 'BIM,Electricity',
 'BIM,Solar',
 'BIM,Wind+power',
 'BIM,Green+infrastructure',
 'BIM,Life+cycle+assessment',
 'Building+Information+Modelling,Green+Building',
 'Building+Information+Modelling,Green+Buildings',
 'Building+Information+Modelling,Sustainable+building',
 'Building+Information+Modelling,Sustainable+buildings',
 'Building+Information+Modelling,Environmentally+friendly+design',
 'Building+Information+Modelling,Sustainability',
 'Building+Information+Modelling,Sustainable+development',
 'Building+Information+Modelling,Water',
 'Building+Information+Modelling,Waste',
 'Building+Information+Modelling,Thermal+comfort',
 'Building+Information+Modelling,Electricity',
 'Building+Information+Modelling,Solar',
 'Building+Information+M

In [7]:
df['journalName'] = journal

In [8]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19,journalName
0,BIM,Energy,https://www.tandfonline.com/doi/full/10.1080/1...,A framework to integrate object-oriented physi...,Abstract,This paper presents a framework for integratin...,Introduction,A lack of integration between building design ...,Literature review,US DOE (2014) has been providing information o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tbps20
1,Building+Information+Modelling,Energy,https://www.tandfonline.com/doi/full/10.1080/1...,Building energy model reduction using model-cl...,Abstract,This paper introduces a novel building energy ...,Introduction,The building sector contributes to 30% of the ...,Methodology,The proposed model reduction method is constru...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tbps20
2,Building+Information+Modelling,Energy,https://www.tandfonline.com/doi/full/10.1080/1...,A framework to integrate object-oriented physi...,Abstract,This paper presents a framework for integratin...,Introduction,A lack of integration between building design ...,Literature review,US DOE (2014) has been providing information o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tbps20


In [9]:
jName = 'kgRawData/KG-' + journal +'.csv'
df.to_csv(jName)

In [10]:
# df = df.where(pd.notnull(df), None)
# df

In [11]:
# df.to_csv('KG-AEDM-7476.csv')

In [12]:
# import re
# app = []
# for i in range(len(tempdf['url'])):
#     text = tempdf['url'][i]
#     pattern = '/abs/'
#     if re.search(pattern, text):
#         app.append(text)
# app

In [13]:
# absIndex = []
# for i in app:
#     for j in range(len(tempdf['url'])):
#         if i == tempdf['url'][j]:
#             absIndex.append(j)
# absIndex

In [14]:
# pos = absIndex

# columns = ['searchTerm1', 'searchTerm2', 'url', 'Title', 'section0',
#        'sectionData0', 'section1', 'sectionData1', 'section2', 'sectionData2',
#        'section3', 'sectionData3', 'section4', 'sectionData4', 'section5',
#        'sectionData5', 'section6', 'sectionData6', 'section7', 'sectionData7',
#        'section8', 'sectionData8', 'section9', 'sectionData9', 'section10',
#        'sectionData10', 'section11', 'sectionData11', 'section12',
#        'sectionData12', 'section13', 'sectionData13', 'section14',
#        'sectionData14', 'section15', 'sectionData15']
# ndf = pd.DataFrame(columns = columns)

# # ndf = ndf.append(tempdf.loc[1].values)
# ndf.loc[1] = tempdf.loc[1].values
# cnt = 0
# for value in absIndex:
#     ndf.loc[cnt] = tempdf.loc[value].values
#     cnt = ant + 1

# tempdf.drop(tempdf.index[pos], inplace=True)


In [1]:
journalName = 'tbps20'

import pandas as pd
df = pd.read_csv('kgRawData/KG-'+ journalName +'.csv')
df = df.where(pd.notnull(df), None)

import re
absRows = []
rowsFound = []
for i in range(len(df['url'])):
    text = df['url'][i]
    pattern = '/abs/'
    if re.search(pattern, text):
        absRows.append(i)
    else:
        rowsFound.append(i)
        
rowsFoundDf = pd.DataFrame()
rowsFoundDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

rowsFoundDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
rowsFoundDf = rowsFoundDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    rowsFoundDf[section] = ''
    rowsFoundDf[sectionData] = ''

    
absRowsDf = pd.DataFrame()
absRowsDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

absRowsDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
absRowsDf = absRowsDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    absRowsDf[section] = ''
    absRowsDf[sectionData] = ''
    
    
for entry in range(len(rowsFound)):
    rowsFoundDf.loc[entry] = df.loc[rowsFound[entry]]

for entry in range(len(absRows)):
    absRowsDf.loc[entry] = df.loc[absRows[entry]]

rowsFoundDf.to_csv('kgRawData/tandf/kg-'+ journalName +'-full.csv')
absRowsDf.to_csv('kgRawData/tandf/kg-'+ journalName +'-abs.csv')

print(len(absRowsDf))
print(len(rowsFoundDf))

0
3
